# Pacotes

In [1]:
import pandas as pd
from dask import dataframe as dd

# ICMS

Capítulo 3: Estatísticas de Referência - TABELA 1.2 - Estado de São Paulo: histórico da receita tributária em valores nominais

In [2]:
df = pd.read_excel(
    'https://portal.fazenda.sp.gov.br/acessoinformacao/Downloads/Relatorios-da-Receita-Tributaria/2020/maio/tab12_Receita_Tribut%C3%A1ria_hist%C3%B3rico_nominal_abr20.xls', 
    skiprows=10,
    index_col=[0], date_parser=True
)
df = df.dropna()
df = df.drop([i for i in range(1995,2021)] + ["1994g"])
df.index = pd.date_range(start="01/01/1994", end='05/01/2020', freq='M')
df = df[["ICMS"]].apply(pd.to_numeric, errors='coerce')
df.columns = ["ICMS Nominal milhões de reais"]
df["ICMS acumulado no ano nominal"] = df.groupby(df.index.to_period('Y')).cumsum(axis=0)
df["Variação % sobre Acumulado nominal ano anterior"] = df[["ICMS acumulado no ano nominal"]].pct_change(12)
df["Variação % sobre mesmo mês ano anterior nominal"] = df[["ICMS Nominal milhões de reais"]].pct_change(12)
#df.to_csv('../data/ICMS_nominal.csv')
df

,ICMS Nominal milhões de reais,ICMS acumulado no ano nominal,Variação % sobre Acumulado nominal ano anterior,Variação % sobre mesmo mês ano anterior nominal
1994-01-31,943.285631,943.285631,NaN,NaN
1994-02-28,736.846597,1680.132228,NaN,NaN
1994-03-31,999.192022,2679.324250,NaN,NaN
1994-04-30,930.323924,3609.648173,NaN,NaN
1994-05-31,891.347049,4500.995222,NaN,NaN
...,...,...,...,...
2019-12-31,14602.358693,146647.102872,0.066424,0.208153
2020-01-31,13082.640610,13082.640610,0.047259,0.047259
2020-02-29,11927.616739,25010.257349,0.047698,0.048181
2020-03-31,12214.580916,37224.838265,0.054980,0.070211


# Índice da Produção Física Industrial do Estado de São Paulo

In [3]:
industrial = pd.read_csv(
    'https://sidra.ibge.gov.br/geratabela?format=us.csv&name=tabela3653.csv&terr=N&rank=-&query=t/3653/n3/all/v/3134/p/all/c544/129314/d/v3134%201/l/v,p,c544%2Bt',
    skiprows=3,
)
#industrial = industrial["São Paulo"]
industrial = industrial[industrial["Unidade da Federação"] == "São Paulo"]
industrial.drop(["Seções e atividades industriais (CNAE 2.0)", "Unidade da Federação"], axis='columns', inplace=True)
industrial = industrial.transpose()
industrial.columns = ["Índice da Produção Física Industrial com ajuste sazonal (Média 2012 = 100)"]
industrial.index = pd.date_range(start="01/01/2002", end='04/01/2020', freq='M')
industrial = industrial.apply(pd.to_numeric, errors='coerce')
industrial

,Índice da Produção Física Industrial com ajuste sazonal (Média 2012 = 100)
2002-01-31,75.7
2002-02-28,78.6
2002-03-31,78.1
2002-04-30,78.2
2002-05-31,78.0
...,...
2019-11-30,84.4
2019-12-31,83.5
2020-01-31,84.6
2020-02-29,83.5


In [4]:
df = df.merge(industrial, left_index=True, right_index=True)
df

,ICMS Nominal milhões de reais,ICMS acumulado no ano nominal,Variação % sobre Acumulado nominal ano anterior,Variação % sobre mesmo mês ano anterior nominal,Índice da Produção Física Industrial com ajuste sazonal (Média 2012 = 100)
2002-01-31,2956.253605,2956.253605,0.082963,0.082963,75.7
2002-02-28,2612.977840,5569.231444,0.124412,0.175304,78.6
2002-03-31,2603.100729,8172.332174,0.042124,-0.098954,78.1
2002-04-30,2964.822876,11137.155049,0.050938,0.076024,78.2
2002-05-31,2848.263981,13985.419030,0.039138,-0.004565,78.0
...,...,...,...,...,...
2019-11-30,12733.549560,132044.744179,0.052766,0.096412,84.4
2019-12-31,14602.358693,146647.102872,0.066424,0.208153,83.5
2020-01-31,13082.640610,13082.640610,0.047259,0.047259,84.6
2020-02-29,11927.616739,25010.257349,0.047698,0.048181,83.5


# Câmbio

In [5]:
url = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.'+ str(3697) +'/dados?formato=csv'
cambio = pd.read_csv(
    url, 
    sep = ';', 
    index_col = 'data', 
    parse_dates=True, 
    decimal= ',', 
    date_parser = lambda x: pd.datetime.strptime(x, '%d/%m/%Y')
  )
cambio.index.name = ''
cambio.columns = ["Taxa de câmbio"]
cambio = cambio["1994":]
cambio.index = pd.date_range(start="01/01/1994", end='05/01/2020', freq='M')
cambio

<ipython-input-5-cc25103ed0b5>:8: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  date_parser = lambda x: pd.datetime.strptime(x, '%d/%m/%Y')


,Taxa de câmbio
1994-01-31,390.8338
1994-02-28,550.7872
1994-03-31,768.1092
1994-04-30,1109.5466
1994-05-31,1585.4550
...,...
2019-12-31,4.1089
2020-01-31,4.1489
2020-02-29,4.3404
2020-03-31,4.8832


In [6]:
df = df.merge(cambio, left_index=True, right_index=True)
df

,ICMS Nominal milhões de reais,ICMS acumulado no ano nominal,Variação % sobre Acumulado nominal ano anterior,Variação % sobre mesmo mês ano anterior nominal,Índice da Produção Física Industrial com ajuste sazonal (Média 2012 = 100),Taxa de câmbio
2002-01-31,2956.253605,2956.253605,0.082963,0.082963,75.7,2.3771
2002-02-28,2612.977840,5569.231444,0.124412,0.175304,78.6,2.4188
2002-03-31,2603.100729,8172.332174,0.042124,-0.098954,78.1,2.3458
2002-04-30,2964.822876,11137.155049,0.050938,0.076024,78.2,2.3196
2002-05-31,2848.263981,13985.419030,0.039138,-0.004565,78.0,2.4796
...,...,...,...,...,...,...
2019-11-30,12733.549560,132044.744179,0.052766,0.096412,84.4,4.1547
2019-12-31,14602.358693,146647.102872,0.066424,0.208153,83.5,4.1089
2020-01-31,13082.640610,13082.640610,0.047259,0.047259,84.6,4.1489
2020-02-29,11927.616739,25010.257349,0.047698,0.048181,83.5,4.3404


# Exportações

In [7]:
url = 'http://www.mdic.gov.br/balanca/bd/comexstat-bd/ncm/EXP_COMPLETA.zip'
url = "../data/" + url.split('/')[-1].split('.')[0] + '.csv'
export = dd.read_csv(
    url, 
    delimiter=';',
    blocksize=64000000 # = 64 Mb chunks
                    )
export = export[export["SG_UF_NCM"] == "SP"]
export = export[["CO_ANO", "CO_MES", "VL_FOB"]]
export = export.groupby(["CO_ANO", "CO_MES"]).sum().compute()
export.columns = ["Exportações FOB"]
export.index = pd.date_range(start="01/01/1997", end='05/01/2020', freq='M')
export.head()

,Exportações FOB
1997-01-31,1211023263
1997-02-28,1138198641
1997-03-31,1277547555
1997-04-30,1430757413
1997-05-31,1437668143


In [8]:
df = df.merge(export, right_index=True, left_index=True)
df.head()

,ICMS Nominal milhões de reais,ICMS acumulado no ano nominal,Variação % sobre Acumulado nominal ano anterior,Variação % sobre mesmo mês ano anterior nominal,Índice da Produção Física Industrial com ajuste sazonal (Média 2012 = 100),Taxa de câmbio,Exportações FOB
2002-01-31,2956.253605,2956.253605,0.082963,0.082963,75.7,2.3771,1262216472
2002-02-28,2612.977840,5569.231444,0.124412,0.175304,78.6,2.4188,1345978624
2002-03-31,2603.100729,8172.332174,0.042124,-0.098954,78.1,2.3458,1443341004
2002-04-30,2964.822876,11137.155049,0.050938,0.076024,78.2,2.3196,1614508483
2002-05-31,2848.263981,13985.419030,0.039138,-0.004565,78.0,2.4796,1491268802


# Importações

In [11]:
url = 'http://www.mdic.gov.br/balanca/bd/comexstat-bd/ncm/IMP_COMPLETA.zip'
url = "../data/" + url.split('/')[-1].split('.')[0] + '.csv'
imports = dd.read_csv(
    url, 
    delimiter=';',
    blocksize=64000000 # = 64 Mb chunks
                    )
imports = imports[imports["SG_UF_NCM"] == "SP"]
imports = imports[["CO_ANO", "CO_MES", "VL_FOB"]]
imports = imports.groupby(["CO_ANO", "CO_MES"]).sum().compute()
imports.columns = ["Importações FOB"]
imports.index = pd.date_range(start="01/01/1997", end='05/01/2020', freq='M')
imports.head()

,Importações FOB
1997-01-31,1316491218
1997-02-28,1945541277
1997-03-31,2380911712
1997-04-30,2481515163
1997-05-31,2227506586


In [12]:
df = df.merge(imports, right_index=True, left_index=True)
df.head()

,ICMS Nominal milhões de reais,ICMS acumulado no ano nominal,Variação % sobre Acumulado nominal ano anterior,Variação % sobre mesmo mês ano anterior nominal,Índice da Produção Física Industrial com ajuste sazonal (Média 2012 = 100),Taxa de câmbio,Exportações FOB,Importações FOB
2002-01-31,2956.253605,2956.253605,0.082963,0.082963,75.7,2.3771,1262216472,1588583185
2002-02-28,2612.977840,5569.231444,0.124412,0.175304,78.6,2.4188,1345978624,1507819585
2002-03-31,2603.100729,8172.332174,0.042124,-0.098954,78.1,2.3458,1443341004,1654236906
2002-04-30,2964.822876,11137.155049,0.050938,0.076024,78.2,2.3196,1614508483,1752896831
2002-05-31,2848.263981,13985.419030,0.039138,-0.004565,78.0,2.4796,1491268802,1666945463


# Exportando dados

In [13]:
df.to_csv('../data/raw_data.csv')